# 3D Trajectory Generation and Visualization Demo

This notebook showcases 3D trajectory generation (stack-of-spirals, 3D cones), their visualization, and the application of hardware constraints.

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

# Add the parent directory to the Python path to allow importing trajgen
# This assumes the notebook is in 'examples' and 'trajgen.py' is in the parent directory
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from trajgen import (
    Trajectory,
    generate_stack_of_spirals_trajectory,
    generate_3d_cones_trajectory,
    display_trajectory,
    constrain_trajectory
)

%matplotlib inline
# For interactive 3D plots, you might prefer:
# %matplotlib widget 
# Ensure you have ipympl installed: pip install ipympl

## Section 1: Stack-of-Spirals Trajectory

### Generation

In [ ]:
num_spirals_per_plane_sos = 8
num_samples_per_spiral_sos = 128
num_planes_sos = 5
plane_fov_sos = 0.25  # meters
total_z_fov_sos = 0.1 # meters
dt_sos = 4e-6 # seconds

sos_traj = generate_stack_of_spirals_trajectory(
    num_spirals_per_plane=num_spirals_per_plane_sos,
    num_samples_per_spiral=num_samples_per_spiral_sos,
    num_planes=num_planes_sos,
    plane_fov_m=plane_fov_sos,
    total_z_fov_m=total_z_fov_sos,
    dt_seconds=dt_sos,
    name="DemoStackOfSpirals"
)

print("--- Stack-of-Spirals Trajectory Summary ---")
if hasattr(sos_traj, 'summary') and callable(sos_traj.summary):
    sos_traj.summary()
else:
    print(f"Name: {sos_traj.name}")
    print(f"Dimensions: {sos_traj.get_num_dimensions()}")
    print(f"Number of points: {sos_traj.get_num_points()}")
    print(f"Duration: {sos_traj.get_duration_seconds() * 1000:.2f} ms")
    print(f"Metadata keys: {list(sos_traj.metadata.keys())}")
    if 'generator_params' in sos_traj.metadata:
        print("  Generator Params:")
        for key, value in sos_traj.metadata['generator_params'].items():
            if key == 'kz_plane_positions_calculated':
                 print(f"    {key}: {np.array(value)}") # Print array nicely
            else:
                 print(f"    {key}: {value}")

### Visualization

In [ ]:
fig_sos = plt.figure(figsize=(8, 7))
ax_sos = display_trajectory(sos_traj, plot_type="3D", figure=fig_sos, max_total_points=5000) 
if ax_sos: # display_trajectory returns None if plotting fails
    ax_sos.set_title("Stack-of-Spirals Trajectory")
plt.show()

## Section 2: 3D Cones Trajectory

### Generation

In [ ]:
num_cones_cones = 12
num_samples_per_cone_cones = 256
max_k_cones = np.pi / (0.2 / 2) # k_max for FOV of 0.2m, resolution implicitly defined by this
cone_angle_cones = 20.0  # degrees
revolutions_cones = 8.0
dt_cones = 4e-6 # seconds

cones_traj = generate_3d_cones_trajectory(
    num_cones=num_cones_cones,
    num_samples_per_cone=num_samples_per_cone_cones,
    max_k_rad_per_m=max_k_cones,
    cone_angle_deg=cone_angle_cones,
    revolutions_per_cone=revolutions_cones,
    dt_seconds=dt_cones,
    name="Demo3DCones"
)

print("--- 3D Cones Trajectory Summary ---")
if hasattr(cones_traj, 'summary') and callable(cones_traj.summary):
    cones_traj.summary()
else:
    print(f"Name: {cones_traj.name}")
    print(f"Dimensions: {cones_traj.get_num_dimensions()}")
    print(f"Number of points: {cones_traj.get_num_points()}")
    print(f"Duration: {cones_traj.get_duration_seconds() * 1000:.2f} ms")
    print(f"Metadata keys: {list(cones_traj.metadata.keys())}")
    if 'generator_params' in cones_traj.metadata:
        print("  Generator Params:")
        for key, value in cones_traj.metadata['generator_params'].items():
            print(f"    {key}: {value}")

### Visualization

In [ ]:
fig_cones = plt.figure(figsize=(8, 7))
ax_cones = display_trajectory(cones_traj, plot_type="3D", figure=fig_cones, max_total_points=5000, plot_style='-')
if ax_cones:
    ax_cones.set_title("3D Cones Trajectory")
plt.show()

## Section 3: Constraining a 3D Trajectory

In [ ]:
# Generate a "demanding" 3D Cones trajectory
demanding_cones_traj = generate_3d_cones_trajectory(
    num_cones=8,
    num_samples_per_cone=512, # More samples
    max_k_rad_per_m=np.pi / (0.22 / 2), # Slightly higher k_max
    cone_angle_deg=30.0, 
    revolutions_per_cone=10.0, # More revolutions
    dt_seconds=2e-6,  # Shorter dwell time to increase gradient/slew demands
    name="Demanding3DCones"
)

print("--- Original Demanding 3D Cones Trajectory ---")
if hasattr(demanding_cones_traj, 'summary') and callable(demanding_cones_traj.summary):
    demanding_cones_traj.summary()
else:
    print(f"Original Max Grad: {demanding_cones_traj.get_max_grad_Tm():.3f} T/m")
    print(f"Original Max Slew: {demanding_cones_traj.get_max_slew_Tm_per_s():.1f} T/m/s")

# Define Constraints
max_grad_constraint = 30.0 / 1000  # T/m (using 30 mT/m)
max_slew_constraint = 120.0  # T/m/s

print(f"\n--- Applying Constraints ---")
print(f"Max Gradient Limit: {max_grad_constraint:.3f} T/m")
print(f"Max Slew Rate Limit: {max_slew_constraint:.1f} T/m/s")

# Apply Constraints
constrained_3d_traj = constrain_trajectory(
    demanding_cones_traj,
    max_gradient_Tm_per_m=max_grad_constraint,
    max_slew_rate_Tm_per_m_per_s=max_slew_constraint
    # dt_seconds will be taken from demanding_cones_traj (2e-6 s)
)

print("\n--- Constrained 3D Cones Trajectory ---")
if hasattr(constrained_3d_traj, 'summary') and callable(constrained_3d_traj.summary):
    constrained_3d_traj.summary()
else:
    print(f"Constrained Max Grad: {constrained_3d_traj.get_max_grad_Tm():.3f} T/m (Limit: {max_grad_constraint:.3f} T/m)")
    print(f"Constrained Max Slew: {constrained_3d_traj.get_max_slew_Tm_per_s():.1f} T/m/s (Limit: {max_slew_constraint:.1f} T/m/s)")


### Visualization of Original and Constrained 3D Trajectories

In [ ]:
fig1 = plt.figure(figsize=(8, 7))
ax1 = display_trajectory(demanding_cones_traj, plot_type="3D", figure=fig1, 
                         title="Original Demanding 3D Trajectory", max_total_points=3000)
plt.show()

fig2 = plt.figure(figsize=(8, 7))
ax2 = display_trajectory(constrained_3d_traj, plot_type="3D", figure=fig2, 
                         title=f"Constrained 3D Trajectory", max_total_points=3000)
plt.show()

## Conclusion

This notebook demonstrated the generation of 3D Stack-of-Spirals and 3D Cones trajectories. It also showed how to visualize these complex 3D trajectories and apply hardware constraints (maximum gradient and slew rate) to them. These tools are essential for designing and simulating advanced MRI pulse sequences.